# Julia Draw Something

In [1]:
using DataFrames
using TensorFlow
using Distributions
using Images

I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:116] Couldn't open CUDA library libcudnn.so.5. LD_LIBRARY_PATH: /usr/local/cuda/lib64
I tensorflow/stream_executor/cuda/cuda_dnn.cc:3459] Unable to load cuDNN DSO
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcurand.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUD

In [2]:
data_set_folder = "/raided/datasets/sketches_png/png/"
list_of_imgs = open(data_set_folder * "filelist.txt")
img_paths = readlines(list_of_imgs)
close(list_of_imgs)

In [19]:
train_paths = Array(String, 17500)
path_index = 1
count = 1
for i = 1:17500
    train_paths[i] = img_paths[path_index]
    path_index += 1
    count += 1
    if count % 71 == 0
        count = 1
        path_index += 10
    end
end

test_paths = Array(String, 2500)
path_index = 71
count = 1
for i = 1:2500
    test_paths[i] = img_paths[path_index]
    path_index += 1
    count += 1
    if count % 11 == 0
        path_index += 70
        count = 1
    end
end

In [20]:
files_in_folder = readdir(data_set_folder)
labels = filter(x->isdir(data_set_folder * x) == true, files_in_folder)

250-element Array{String,1}:
 "airplane"     
 "alarm clock"  
 "angel"        
 "ant"          
 "apple"        
 "arm"          
 "armchair"     
 "ashtray"      
 "axe"          
 "backpack"     
 "banana"       
 "barn"         
 "baseball bat" 
 ⋮              
 "umbrella"     
 "van"          
 "vase"         
 "violin"       
 "walkie talkie"
 "wheel"        
 "wheelbarrow"  
 "windmill"     
 "wine-bottle"  
 "wineglass"    
 "wrist-watch"  
 "zebra"        

In [21]:
x = TensorFlow.placeholder(Float32)
y_ = TensorFlow.placeholder(Float32)
W = get_variable("weights", [77841, 250], Float32)
b = get_variable("bias", [250], Float32)

LoadError: Tensorflow error: Status: Duplicate node name in graph: 'weights'


In [22]:
y = nn.softmax(x*W + b)
cross_entropy = reduce_mean(-reduce_sum(y_ .* log(y), reduction_indices=[2]))
train_step = train.minimize(train.GradientDescentOptimizer(.00001), cross_entropy)

<Tensor NoOp_3:1 shape=unknown dtype=?>

In [23]:
type DataLoader
    cur_id::Int
    order::Vector{Int}
end

TrainLoader() = DataLoader(1, shuffle(1:17500))
TestLoader() = DataLoader(1, shuffle(1:2500))

TestLoader (generic function with 1 method)

In [24]:
function next_batch(loader::DataLoader, batch_size)
    x = zeros(Float32, batch_size, 77841)
    y = zeros(Float32, batch_size, 250)
    for i = 1:batch_size
        path = train_paths[loader.order[loader.cur_id]][1:end-1]
#         println(path)
        sketch_name = split(path, "/")[1]
        img = load(data_set_folder * path)
        img = restrict(restrict(img))
        flatten_img = reshape(img, 77841)
        
        x[i, :] = flatten_img
        label = find((x -> x == sketch_name), labels)[1]
        y[i, label] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 17500
            break
        end
    end
    x, y
end

next_batch (generic function with 1 method)

In [25]:
function get_test(loader::DataLoader)
    x = zeros(Float32, 2500, 77841)
    y = zeros(Float32, 2500, 250)
    for i = 1:2500
        path = test_paths[loader.order[loader.cur_id]][1:end-1]
        sketch_name = split(path, "/")[1]
        img = load(data_set_folder * path)
        img = restrict(restrict(img))
        flatten_img = reshape(img, 77841)
        
        x[i, :] = flatten_img
        label = find((x -> x == sketch_name), labels)[1]
        y[i, label] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 2500
            println("========================")
            println(sketch_name)
            break
        end
    end
    x, y
end

get_test (generic function with 1 method)

In [26]:
loader = TrainLoader()
sess = Session()
run(sess, initialize_all_variables())
for i in 1:875
    batch = next_batch(loader, 20)
    run(sess, train_step, Dict(x=>batch[1], y_=>batch[2]))
    println("DONE BATCH ", i)
end

DONE BATCH 1
DONE BATCH 

I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0)


2
DONE BATCH 3
DONE BATCH 4
DONE BATCH 5
DONE BATCH 6
DONE BATCH 7
DONE BATCH 8


Error encountered while loading "/raided/datasets/sketches_png/png/lightbulb/9709.png".
Fatal error:


LoadError: InterruptException:

In [ ]:
correct_prediction = indmax(y, 2) .== indmax(y_, 2)
accuracy=reduce_mean(cast(correct_prediction, Float64))
test_loader = TestLoader()
testx, testy = get_test(test_loader)

println(run(sess, indmax(y, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, indmax(y_, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, accuracy, Dict(x=>testx, y_=>testy)))

In [12]:
function sketch_input(path, sketch_name)
    x = zeros(Float32, 1, 77841)
    y = zeros(Float32, 1, 250)
    img = load(path)
    img = restrict(restrict(img))
    flatten_img = reshape(img, 77841)
    x[1, :] = flatten_img
    
    label = find((x -> x == sketch_name), labels)[1]
    y[1, label] = 1.0
    
    x, y
end

sketch_input (generic function with 1 method)

In [13]:
sketch_path = "/raided/datasets/sketches_png/png/axe/711.png"
sketch_label = "axe"

test_img, test_label = sketch_input(sketch_path, sketch_label)
what_it_thinks = labels[run(sess, indmax(y,2), Dict(x=>test_img, y_=>test_label))[1] + 1]
println("Is it an " * what_it_thinks * "? Answer: ", (sketch_label == what_it_thinks), ".")

Is it an foot? Answer: false.
